### Run Triton Server

In [ ]:
!docker run --rm -p 8000:8000 -p 8001:8001 -p 8002:8002 -v /home/sachin/Documents/OXFORD/Project\ 15/Code/Model\ Training/models/:/models nvcr.io/nvidia/tritonserver:20.11-py3 tritonserver --model-repository=/models


== Triton Inference Server ==

NVIDIA Release 20.11 (build <unknown>)

Copyright (c) 2018-2020, NVIDIA CORPORATION.  All rights reserved.

Various files include modifications (c) NVIDIA CORPORATION.  All rights reserved.
NVIDIA modifications are covered by the license terms that apply to the underlying
project or file.

   Use 'nvidia-docker run' to start this container; see
   https://github.com/NVIDIA/nvidia-docker/wiki/nvidia-docker .

NOTE: The SHMEM allocation limit is set to the default of 64MB.  This may be
   insufficient for the inference server.  NVIDIA recommends the use of the following flags:
   nvidia-docker run --shm-size=1g --ulimit memlock=-1 --ulimit stack=67108864 ...

E0124 09:36:14.433448 1 pinned_memory_manager.cc:196] failed to allocate pinned system memory: CUDA driver version is insufficient for CUDA runtime version
I0124 09:36:14.434543 1 model_repository_manager.cc:810] loading: gunshot-detection:1
I0124 09:36:14.434697 1 model_repository_manager.cc:810] loa

I0124 09:36:14.883347 1 http_server.cc:2736] Started Metrics Service at 0.0.0.0:8002


In [18]:
import tritonclient.http as httpclient
triton_client = httpclient.InferenceServerClient(url="localhost:8000")
triton_client.get_model_metadata("gunshot-detection")

{'name': 'gunshot-detection',
 'versions': ['1'],
 'platform': 'tensorflow_savedmodel',
 'inputs': [{'name': 'flatten_input',
   'datatype': 'FP32',
   'shape': [-1, 10, 13]}],
 'outputs': [{'name': 'dense_3', 'datatype': 'FP32', 'shape': [-1, 2]}]}

In [19]:
import json
import numpy as np
from sklearn.model_selection import train_test_split

# path to json file that stores MFCCs and genre labels for each processed segment
DATA_PATH = "./dataset.json"


def load_data(data_path):
    """Loads training dataset from json file.

        :param data_path (str): Path to json file containing data
        :return X (ndarray): Inputs
        :return y (ndarray): Targets
    """

    with open(data_path, "r") as fp:
        data = json.load(fp)

    # convert lists to numpy arrays
    X = np.array(data["mfcc"])
    y = np.array(data["labels"])

    print("Data succesfully loaded!")
    return X, y


# load data
X, y = load_data(DATA_PATH)
# create train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Data succesfully loaded!


In [33]:
import numpy as np
inputs = []
outputs = []
inputs.append(httpclient.InferInput('flatten_input', [1,10,13], "FP32"))

inputs[0].set_data_from_numpy(np.float32(X_test[0:1]))
outputs.append(httpclient.InferRequestedOutput('dense_3',binary_data=False))
results = triton_client.infer("gunshot-detection",inputs,outputs=outputs)

prediction_data = results.get_response()
prediction = prediction_data.get("outputs")[0]["data"]
print(prediction)

# get index with max value
predicted_index = np.argmax(prediction)

print("Target: {}, Predicted label: {}".format(y_test[0:1], predicted_index))

[1.1463476563365724e-16, 1.0]
Target: [1], Predicted label: 1
